📢 [8월 19일 과제] Output Parser & 모델 관리 개념·코드 복습
오늘은 LLM의 출력을 구조화된 형식으로 다루는 방법과, 모델 관리 관련 기능을 실습했습니다.
이번 과제는 단순 텍스트 응답이 아닌, 데이터 형태를 변환·제어하는 방법을 연습하고 이해하는 것이 목표입니다.



📄 정리 목차
오늘 학습한 주요 Output Parser 및 모델 관리 기능 간단 요약 (아래 단어들은 예시입니다)
- PydanticOutputParser
- JsonOutputParser
- PandasDataFrameOutputParser
- EnumOutputParser
- OutputFixingParser
- StructuredOutputParser
- Chat-Models / Cache / Model Serialization

교재의 03-OutputParser 코드의 프롬프트 수정 후 다시 실습 + 코드 주석 달기

실습 과제  (한 번에 모든 OutputParser를 공부하기 보다는 자신이 흥미 있거나 목표하는 것 위주로 선택하는 것을 추천 드립니다! )
- 메일 요약 → 뉴스 요약으로 변경
- 식당 리뷰 감정 분석  →  긍정/부정 등 카테고리화
- 쇼핑몰 주문 내역 → 표 정리
- 운동 기록 정리 → 딕셔너리 형태로 출력
- 책 추천 목록 생성 → JSON 리스트 형태로 출력

## PydanticOutputParser

이 클래스를 활용하면 언어 모델의 출력을 특정 데이터 모델에 맞게 변환하여 정보를 더 쉽게 처리하고 활용할 수 있습니다.

## 주요 메서드

`PydanticOutputParser` (대부분의 OutputParser에 해당)에는 주로 **두 가지 핵심 메서드**가 구현되어야 합니다.

- **`get_format_instructions()`**: 언어 모델이 출력해야 할 정보의 형식을 정의하는 지침을 제공합니다. 예를 들어, 언어 모델이 출력해야 할 데이터의 필드와 그 형태를 설명하는 지침을 문자열로 반환할 수 있습니다. 이 지침은 언어 모델이 출력을 구조화하고 특정 데이터 모델에 맞게 변환하는 데 매우 중요합니다.
- **`parse()`**: 언어 모델의 출력(문자열로 가정)을 받아 이를 특정 구조로 분석하고 변환합니다. Pydantic과 같은 도구를 사용하여 입력된 문자열을 사전 정의된 스키마에 따라 검증하고, 해당 스키마를 따르는 데이터 구조로 변환합니다.

In [5]:
# 실시간 출력을 위한 import
from langchain_teddynote.messages import stream_response

In [4]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field


llm = ChatOpenAI(temperature=0, model_name="gpt-4.1")

### 실습1: 영화 후기 요약

In [ ]:
# Pydantic 모델을 정의합니다.
# 이 모델은 LLM이 생성해야 할 출력의 구조를 명시합니다.
class ReviewSummary(BaseModel):
    """영화 후기 요약 모델"""

    title: str = Field(description="영화의 제목")
    summary: str = Field(description="영화 감상을 200자 이내로 요약한 내용")
    actors: str = Field(description="출연 배우 이름")
    keywords: str = Field(description="영화 핵심 키워드")

In [ ]:
# PydanticOutputParser를 초기화합니다.
# 이 파서는 Pydantic 모델에 정의된 스키마에 따라 LLM의 출력을 파싱합니다.
parser = PydanticOutputParser(pydantic_object=ReviewSummary)
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"description": "영화 후기 요약 모델", "properties": {"title": {"description": "영화의 제목", "title": "Title", "type": "string"}, "summary": {"description": "영화 감상을 200자 이내로 요약한 내용", "title": "Summary", "type": "string"}, "actors": {"description": "출연 배우 이름", "title": "Actors", "type": "string"}, "keywords": {"description": "영화 핵심 키워드", "title": "Keywords", "type": "string"}}, "required": ["title", "summary", "actors", "keywords"]}
```


In [ ]:
# 프롬프트 템플릿을 생성합니다.
# `format_instructions` 변수는 파서가 요구하는 출력 형식을 동적으로 삽입합니다.
prompt_template = """
당신은 영화 후기를 읽고 아래에 주어진 형식에 맞게 정보를 추출하여 요약하는 전문가입니다.
주어진 형식에 맞게 JSON을 출력해야 합니다.

{format_instructions}

다음은 요약할 영화 후기입니다:
{review_article}
"""

# PromptTemplate 객체를 생성하고, 필요한 변수(format_instructions, review_article)를 설정합니다.
prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["review_article"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)
prompt

PromptTemplate(input_variables=['review_article'], input_types={}, partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"description": "영화 후기 요약 모델", "properties": {"title": {"description": "영화의 제목", "title": "Title", "type": "string"}, "summary": {"description": "영화 감상을 200자 이내로 요약한 내용", "title": "Summary", "type": "string"}, "actors": {"description": "출연 배우 이름", "title": "Actors", "type": "string"}, "keywords": {"description": "영화 핵심 키워드", "title": "Keywords", "type": "string"}}, "required": ["title", "summary", "actors", "keywords"]}\n```'}, 

In [ ]:
# 예제 영화 후기입니다.
example_review_article = """
내 가족이 좀비가 된다면? 인간이 아닌 좀비가 나를 물어 뜯으려고 한다면, 어떻게 할까. 여기 좀비가 된 딸을 훈련시키며 키우는 아빠가 있다. 영화 '좀비딸'(감독 필감성)이 웃음과 눈물을 장착하고 출격 준비를 마쳤다.

'좀비딸'은 이 세상 마지막 남은 좀비가 된 딸을 지키기 위해 극비 훈련에 돌입한 딸바보 아빠의 코믹 드라마. 웹툰 원작의 '좀비딸'은 원작에 충실하면서도 관객들이 좋아할만한 드라마틱한 요소들을 잘 살려냈다.

딸과 함께 살고 있는 정환(조정석 분)은 딸 수아(최유리 분)과 친구처럼 지내는 다정한 아빠다. 중학생 딸의 생일날, 집에서 둘만의 조촐한 파티를 하고 있는 가운데 갑자기 좀비 바이러스가 동네를 덮친다. 정환과 수아는 좀비 바이러스를 피해 할머니 밤순이 있는 은봉리로 가기로 한다. 하지만 두 사람이 차를 타러 가는 과정에 수아가 좀비에게 물리고, 자동차 안에서 수아도 좀비로 변신하면서 이야기가 본격적으로 시작된다.

정환은 좀비가 된 딸이 사살당하는 것을 막기 위해 좀비딸을 훈련시킨다. 오로지 사랑의 힘으로 좀비딸을 훈련시키는 정환과 수아의 이야기가 만화적인 스토리들이 관객을 웃기고 감정을 건드리고 눈물을 터뜨린다.

'좀비딸'은 놀랍거나 새로운 이야기는 아니다. 이제는 관객에게 익숙한 좀비물에 코믹을 묻혔다. 부성애라는 코드 역시 흔히 볼 수 있는 이야기지만 캐릭터를 재대로 입은 조정석이 보여주는 '딸 아빠'는 통할수 밖에. 맹수 사육사 정환은 좀비를 마치 애완동물이라도 되는 듯이 교육한다. 딸을 보면 마음 약해지는 아빠 정환 뒤로는 좀비 손녀를 '효자손' 맴매로 다스리는 할머니 밤순이 있다. 할머니의 효자손에 한 대 맞은 뒤로부터 효자손만 보면 주춤하는 좀비 손녀는 할머니, 아빠와 새로운 케미를 보여준다. 세 사람이 보여주는 케미 위로 다양한 인물들이 등장해 양념을 더해준다.

중후반부에 삐끗하는 장면도 있다. 이야기를 만들다보니 세련되지 않는 이야기들도 등장하지만 착착 이어진 서사에 크게 걸리지 않고 넘어 간다. 그러다가 영화 말미 관객의 눈물샘을 터뜨린다. 웃다가 울면서 끝나니, 영화를 보고 나면 애정이 생기게 된다, 다 아는 아빠와 딸의 이야기도 조정석을 통해 나오면 더 재밌다.

조정석은 정환 그 자체다. 자신이 잘하는 연기를 십분 활용해서 영화를 이끈다. 실제 딸 아빠인 조정석은 "감정이 과하게 넘치지 않도록 노력했다"라고 말할만큼, 좀비가 된 딸을 살리기 위해 노력하는 모습을 보여주며 능청스럽게 감동을 전한다. 딸과 함께 있을때는 노력하는 단단한 아빠였던 그가, 딸의 옷을 가지러 집에 갔다가 평범했던 일상을 떠올리며 무너지는 눈물 연기에는 그의 진심이 담겼다. 최유리는 변화하는 좀비 연기를 표현하며 귀엽게, 때로는 제대로 각잡힌 좀비 연기를 보여준다. 캐릭터를 연구하고 연습한 마음이 느껴진다.

이정은은 이 영화의 '킥'이다. 이정은은 실제 나이보다 훨씬 많은 할머니 연기를 너무나 자연스럽게, 살아있게 펼쳤다. 역시 연기 내공이 느껴진다. 아빠와 좀비딸의 가운데서 균형을 잡아주며 영화를 맛깔나게 만든다. 윤경호, 조여정 등 정환의 친구 역할을 맡은 두 사람도 제 몫을 해내며 서사에 힘들 보탠다.

고양이 '애용이'도 '좀비딸'의 큰 매력이다. 어찌나 연기를 잘하는지. 사랑스러운 매력이 영화 시작부터 관객을 녹인다.

'좀비딸'은 요즘 홍수처럼 쏟아지는 장르물이나 시리즈처럼 스타일리시한 이야기는 아니다. 오히려 조금 촌스럽고 그래서 정겹다. 방학때 해외로 여행을 떠나는 대신, 할머니집에 가서 모깃불 피워놓고 옥수수를 삶아먹는 느낌이랄까. 범죄물처럼 때리고 부수는 장면이 있는 것도 아니고 거대한 세계관을 새로 창조하는 이야기도 아니다. 일상에 좀비 한 방울을 떨어뜨리고 그 속에서 고군분투하는 사랑스러운 캐릭터들이 영화를 누빈다.

영화 '엑시트', '파일럿'을 연이어 흥행시키며 여름 극장가 구원투수로 자리잡은 조정석은 이번에 '좀비딸'을 통해 3연타를 노린다. 더운 여름, 가족과 함께 볼 영화로 이만한 영화가 없다. 연인과 친구와 함께 극장에 들어가서 본다며 웃으면서 혹은 서로 눈물 닦을 티슈를 챙겨주며 경쾌하게 볼 수 있을것 같다.
"""

In [ ]:
# 프롬프트, 모델, 파서를 체인으로 연결합니다.
chain = prompt | llm | parser

# 체인을 실행하고 요약 결과를 얻습니다.
# `invoke` 메서드는 체인을 실행하여 최종 결과를 반환합니다.
# PydanticOutputParser 덕분에 결과는 NewsSummary 객체 형태로 반환됩니다.
summary_result = chain.invoke({"review_article": example_review_article})
summary_result

ReviewSummary(title='좀비딸', summary='좀비가 된 딸을 사랑으로 훈련시키는 아빠의 이야기를 코믹하면서도 감동적으로 그린 영화다. 익숙한 좀비물에 부성애와 가족애를 더해 웃음과 눈물을 선사하며, 배우들의 연기와 케미가 돋보인다. 일상에 좀비를 녹여낸 정겹고 따뜻한 작품이다.', actors='조정석, 최유리, 이정은, 윤경호, 조여정', keywords='좀비, 부성애, 가족, 코믹 드라마, 감동, 훈련, 케미')

In [ ]:
print("--- 객체 정보 ---")
print(f"타입: {type(summary_result)}")
print(f"내용: {summary_result}")

--- 객체 정보 ---
타입: <class '__main__.ReviewSummary'>
내용: title='좀비딸' summary='좀비가 된 딸을 사랑으로 훈련시키는 아빠의 이야기를 코믹하면서도 감동적으로 그린 영화다. 익숙한 좀비물에 부성애와 가족애를 더해 웃음과 눈물을 선사하며, 배우들의 연기와 케미가 돋보인다. 일상에 좀비를 녹여낸 정겹고 따뜻한 작품이다.' actors='조정석, 최유리, 이정은, 윤경호, 조여정' keywords='좀비, 부성애, 가족, 코믹 드라마, 감동, 훈련, 케미'


In [ ]:
# 결과 출력
print("--- 요약 결과 ---")
print(f"제목: {summary_result.title}")
print(f"후기 요약: {summary_result.summary}")
print(f"출연 배우: {summary_result.actors}")
print(f"키워드: {summary_result.keywords}")

--- 요약 결과 ---
제목: 좀비딸
후기 요약: 좀비가 된 딸을 사랑으로 훈련시키는 아빠의 이야기를 코믹하면서도 감동적으로 그린 영화다. 익숙한 좀비물에 부성애와 가족애를 더해 웃음과 눈물을 선사하며, 배우들의 연기와 케미가 돋보인다. 일상에 좀비를 녹여낸 정겹고 따뜻한 작품이다.
출연 배우: 조정석, 최유리, 이정은, 윤경호, 조여정
키워드: 좀비, 부성애, 가족, 코믹 드라마, 감동, 훈련, 케미


## CommaSeparatedListOutputParser

`CommaSeparatedListOutputParser`는 쉼표로 구분된 항목 목록을 반환할 필요가 있을 때 유용한 출력 파서입니다.

이 파서를 사용하면, 입력된 데이터나 요청된 정보를 쉼표로 구분하여 명확하고 간결한 목록 형태로 제공할 수 있습니다. 예를 들어, 여러 개의 데이터 포인트, 이름, 항목 또는 다양한 값을 나열할 때 효과적으로 정보를 정리하고 사용자에게 전달할 수 있습니다.

이 방법은 정보를 구조화하고 가독성을 높이며, 특히 데이터를 다루거나 리스트 형태의 결과를 요구하는 경우에 매우 유용합니다.

In [ ]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser

# 콤마로 구분된 리스트 출력 파서 초기화
output_parser = CommaSeparatedListOutputParser()

# 출력 형식 지침 가져오기
format_instructions = output_parser.get_format_instructions()
# 프롬프트 템플릿 설정
prompt = PromptTemplate(
    # 제시한 요리의 재료를 나열하라는 템플릿
    template="요리 {cook}의 재료를 나열하세요.\n{format_instructions}",
    input_variables=["cook"],  # 입력 변수로 'cook' 사용
    # 부분 변수로 형식 지침 사용
    partial_variables={"format_instructions": format_instructions},
)

# ChatOpenAI 모델 초기화
model = ChatOpenAI(temperature=0.3, api_key=OPENAI_API_KEY)

# 프롬프트, 모델, 출력 파서를 연결하여 체인 생성
chain = prompt | model | output_parser

### 실습2: 요리 이름을 입력하면 요리에 필요한 재료를 쉼표 구분으로 출력

In [ ]:
# "된장찌개"에 대한 체인 호출 실행
chain.invoke({"cook": "된장찌개"})

['된장',
 '물',
 '두부',
 '양파',
 '대파',
 '청양고추',
 '마늘',
 '돼지고기',
 '대파',
 '감자',
 '무',
 '김치',
 '고추장']

In [ ]:
# "된장찌개"에 대한 체인 호출 실행
chain.invoke({"cook": "감바스"})

['새우', '올리브 오일', '마늘', '파슬리', '레몬 주스', '소금', '후추', '파프리카 분말']

In [ ]:
# "된장찌개"에 대한 체인 호출 실행
chain.invoke({"cook": "마파두부"})

['두부',
 '고기(돼지고기 또는 소고기)',
 '두반장',
 '고추기름',
 '대파',
 '마늘',
 '생강',
 '고추가루',
 '후추',
 '설탕',
 '간장',
 '물',
 '식용유']

# StructuredOutputParser

StructuredOutputParser는 LLM에 대한 답변을 `dict` 형식으로 구성하고, key/value 쌍으로 여러 필드를 반환하고자 할 때 유용하게 사용할 수 있습니다.

## 장점
Pydantic/JSON 파서가 더 강력하다는 평가를 받지만, StructuredOutputParser는 로컬 모델과 같은 덜 강력한 모델에서도 유용합니다. 이는 GPT나 Claude 모델보다 인텔리전스가 낮은(즉, parameter 수가 적은) 모델에서 특히 효과적입니다.

## 참고 사항
로컬 모델의 경우 `Pydantic` 파서가 동작하지 않는 상황이 빈번하게 발생할 수 있습니다. 이러한 경우, 대안으로 StructuredOutputParser를 사용하는 것이 좋은 해결책이 될 수 있습니다.


In [ ]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

### 실습3: 사용자가 장소를 입력하면 해당 장소의 다양한 정보와 출처를 출력

In [ ]:
# 사용자의 질문에 대한 답변
response_schemas = [
    ResponseSchema(name="question", description="사용자의 질문한 장소"),
    ResponseSchema(name="answer", description="사용자의 질문한 장소에 대한 간단한 소개"),
    ResponseSchema(name="where", description="사용자가 질문한 장소의 주소"),
    ResponseSchema(name="visit", description="사용자가 질문한 장소로 가는 방법"),
    ResponseSchema(name="when", description="사용자가 질문한 장소의 운영 시간 및 정기휴일 정보"),
    ResponseSchema(name="source", description="사용자의 질문에 답하기 위해 사용된 `출처`, `웹사이트주소` 이여야 합니다.", )
]
# 응답 스키마를 기반으로 한 구조화된 출력 파서 초기화
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [ ]:
# 출력 형식 지시사항을 파싱합니다.
format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    # 사용자의 질문에 최대한 답변하도록 템플릿을 설정합니다.
    template="사용자가 물어본 장소에 대해서 알려주세요. \n{format_instructions}\n{question}",
    # 입력 변수로 'question'을 사용합니다.
    input_variables=["question"],
    # 부분 변수로 'format_instructions'을 사용합니다.
    partial_variables={"format_instructions": format_instructions},
)

In [ ]:
chain = prompt | model | output_parser  # 프롬프트, 모델, 출력 파서를 연결

In [ ]:
# 경복궁에 대해서 질문합니다.
chain.invoke({"question": "경복궁"})

{'question': '경복궁',
 'answer': '경복궁은 조선 시대의 궁궐로, 서울의 대표적인 관광지이며 국보 1호로 지정되어 있는 역사적인 장소입니다.',
 'where': '161 Sajik-ro, Sejongno, Jongno-gu, Seoul, South Korea',
 'visit': '지하철 3호선 안국역 2번 출구에서 도보로 약 5분 소요',
 'when': '매주 화요일은 휴관이며, 운영 시간은 9:00 - 18:00 입니다.',
 'source': '한국관광공사, https://english.visitkorea.or.kr'}

In [ ]:
# 서울 타워에 대해서 질문합니다.
chain.invoke({"question": "서울 타워"})

{'question': '서울 타워',
 'answer': '서울의 대표적인 랜드마크이자 전망대로 유명한 관광지',
 'where': '105 Namsangongwon-gil, Yongsan 2(i)ga-dong, Yongsan-gu, Seoul, South Korea',
 'visit': '지하철 4호선 숙대입구역 9번 출구에서 도보로 약 15분 소요',
 'when': '매일 10:00 - 23:00, 정기휴일 없음',
 'source': '위키피디아, https://www.nseoultower.co.kr/'}

In [ ]:
# 여의도 더현대에 대해서 질문합니다.
chain.invoke({"question": "여의도 더현대"})

{'question': '여의도 더현대',
 'answer': '여의도 더현대는 서울 여의도에 위치한 상업시설로 쇼핑, 식사, 엔터테인먼트 등 다양한 즐길거리를 제공합니다.',
 'where': '서울특별시 영등포구 여의도동 23-1',
 'visit': '지하철 5호선 여의도역 3번 출구에서 도보로 약 5분 소요',
 'when': '매일 10:30 - 22:00, 정기휴일 정보는 사이트를 참고하세요.',
 'source': '공식 홈페이지: http://www.yeouidohyundai.co.kr/'}

## JsonOutputParser

JsonOutputParser는 사용자가 원하는 JSON 스키마를 지정할 수 있게 해주는 도구입니다. 이 도구는 Large Language Model (LLM)이 데이터를 조회하고 결과를 도출할 때, 지정된 스키마에 맞게 JSON 형식으로 데이터를 반환할 수 있도록 설계되었습니다.

LLM이 데이터를 정확하고 효율적으로 처리하여 사용자가 원하는 형태의 JSON을 생성하기 위해서는, 모델의 용량(예: 인텔리전스)이 충분히 커야 합니다. 예를 들어, llama-70B 모델은 llama-8B 모델보다 더 큰 용량을 가지고 있어 보다 복잡한 데이터를 처리하는 데 유리합니다.

**[참고]**

`JSON (JavaScript Object Notation)` 은 데이터를 저장하고 구조적으로 전달하기 위해 사용되는 경량의 데이터 교환 포맷입니다. 웹 개발에서 매우 중요한 역할을 하며, 서버와 클라이언트 간의 통신을 위해 널리 사용됩니다. JSON은 읽기 쉽고, 기계가 파싱하고 생성하기 쉬운 텍스트를 기반으로 합니다.

JSON의 기본 구조
JSON 데이터는 이름(키)과 값의 쌍으로 이루어져 있습니다. 여기서 "이름"은 문자열이고, "값"은 다양한 데이터 유형일 수 있습니다. JSON은 두 가지 기본 구조를 가집니다:

- 객체: 중괄호 {}로 둘러싸인 키-값 쌍의 집합입니다. 각 키는 콜론 :을 사용하여 해당하는 값과 연결되며, 여러 키-값 쌍은 쉼표 ,로 구분됩니다.
- 배열: 대괄호 []로 둘러싸인 값의 순서 있는 목록입니다. 배열 내의 값은 쉼표 ,로 구분됩니다.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser

In [ ]:
# OpenAI 객체를 생성합니다.
model = ChatOpenAI(temperature=0.3, model_name="gpt-4.1-mini", api_key=OPENAI_API_KEY)

### 실습3: 식재료를 입력하면 해당 식재료의 카테고리와 권장 소비기간, 보관 방법을 출력

In [ ]:
from ast import Num
# 원하는 데이터 구조를 정의합니다.
class Ingredient(BaseModel):
    name: str = Field(description="식재료의 이름")
    category: str = Field(description="식재료의 카테고리")
    storageDesc: str = Field(description="식재료의 권장 소비기간 정보")
    storageDays: int = Field(description="식재료의 권장 소비기간 일수 숫자만")
    storageMethod: str = Field(description="식재료의 보관 방법 정보")

In [ ]:
# 파서를 설정하고 프롬프트 템플릿에 지시사항을 주입합니다.
parser = JsonOutputParser(pydantic_object=Ingredient)
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"name": {"description": "식재료의 이름", "title": "Name", "type": "string"}, "category": {"description": "식재료의 카테고리", "title": "Category", "type": "string"}, "storageDesc": {"description": "식재료의 권장 소비기간 정보", "title": "Storagedesc", "type": "string"}, "storageDays": {"description": "식재료의 권장 소비기간 일수 숫자만", "title": "Storagedays", "type": "integer"}, "storageMethod": {"description": "식재료의 보관 방법 정보", "title": "Storagemethod", "type": "string"}}, "required": ["name", "category", "storageDesc", "storageDays", "storageMethod"]}
```


In [ ]:
# 프롬프트 템플릿을 설정합니다.
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 친절한 AI 어시스턴트 입니다. 질문한 식재료에 대한 정보를 알려주세요."),
        ("user", "#Format: {format_instructions}\n\n#Question: {question}"),
    ]
)

prompt = prompt.partial(format_instructions=parser.get_format_instructions())

# 체인을 구성합니다.
chain = prompt | model | parser

question = "당근"

# 체인을 호출하여 쿼리 실행
answer = chain.invoke({"question": question})

In [ ]:
# 타입을 확인합니다.
type(answer)

dict

In [ ]:
# answer 객체를 출력합니다.
answer

{'name': '당근',
 'category': '채소',
 'storageDesc': '냉장 보관 시 약 7~10일 이내에 소비하는 것이 좋습니다.',
 'storageDays': 10,
 'storageMethod': '종이 타월로 감싸서 비닐봉지에 넣어 냉장고 야채 칸에 보관하세요.'}

In [ ]:
question = "감자"

# 체인을 호출하여 쿼리 실행
answer = chain.invoke({"question": question})
answer

{'name': '감자',
 'category': '채소',
 'storageDesc': '서늘하고 건조한 곳에서 보관하면 약 30일간 신선도를 유지할 수 있습니다.',
 'storageDays': 30,
 'storageMethod': '직사광선을 피하고 통풍이 잘 되는 서늘하고 건조한 곳에 보관하세요. 냉장 보관 시 전분이 변해 맛이 떨어질 수 있습니다.'}

In [ ]:
question = "토마토"

# 체인을 호출하여 쿼리 실행
answer = chain.invoke({"question": question})
answer

{'name': '토마토',
 'category': '채소',
 'storageDesc': '냉장 보관 시 5~7일 내에 소비하는 것이 좋습니다.',
 'storageDays': 7,
 'storageMethod': '상온에서는 익으면 빨리 소비하고, 냉장 보관 시 신문지에 싸서 보관하면 신선도를 유지할 수 있습니다.'}

# PandasDataFrameOutputParser

**Pandas DataFrame**은 Python 프로그래밍 언어에서 널리 사용되는 데이터 구조로, 데이터 조작 및 분석을 위한 강력한 도구입니다. DataFrame은 구조화된 데이터를 효과적으로 다루기 위한 포괄적인 도구 세트를 제공하며, 이를 통해 데이터 정제, 변환 및 분석과 같은 다양한 작업을 수행할 수 있습니다.

이 **출력 파서**는 사용자가 임의의 Pandas DataFrame을 지정하여 해당 DataFrame에서 데이터를 추출하고, 이를 형식화된 사전(dictionary) 형태로 조회할 수 있게 해주는 LLM(Large Language Model) 기반 도구입니다.


In [6]:
import pprint
from typing import Any, Dict

import pandas as pd
from langchain.output_parsers import PandasDataFrameOutputParser

In [7]:
# OpenAI 객체를 생성합니다. (gpt-3.5-turbo 모델 사용을 권장)
model = ChatOpenAI(temperature=0.3, model_name="gpt-4.1")

In [8]:
# 출력 목적으로만 사용됩니다.
def format_parser_output(parser_output: Dict[str, Any]) -> None:
    # 파서 출력의 키들을 순회합니다.
    for key in parser_output.keys():
        # 각 키의 값을 딕셔너리로 변환합니다.
        parser_output[key] = parser_output[key].to_dict()
    # 예쁘게 출력합니다.
    return pprint.PrettyPrinter(width=4, compact=True).pprint(parser_output)

### 실습4: 식재료와 권장 소비기간, 보관정보가 저장되어 있는 csv 파일을 padas로 읽어들이기

In [9]:
# 원하는 Pandas DataFrame을 정의합니다.
df = pd.read_csv("data/storageInfo.csv")
df.head()

,category,name,storageDays,storageDesc,storageMethod
0,기타,두부,7,5~7일,물에 담가 밀폐용기에 넣어 냉장 보관. 매일 물을 갈아주면 더 오래 보관 가능합니다.
1,야채,콩나물,5,3~5일,씻지 않은 상태로 물기를 제거하고 밀폐용기에 담아 냉장 보관하세요. 빛을 차단하면 ...
2,야채,숙주나물,4,2~4일,"콩나물보다 쉽게 무르므로, 씻지 않고 밀폐용기에 담아 냉장 보관 후 가급적 빨리 소..."
3,야채,고구마,30,2~4주,10~15°C의 서늘하고 어두운 곳에 신문지로 감싸 보관하세요. (냉장 보관 시 맛...
4,야채,감자,30,2~4주,빛이 없는 서늘하고 통풍이 잘 되는 곳에 보관하세요. 사과와 함께 두면 싹 나는 것...


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   category       60 non-null     object
 1   name           60 non-null     object
 2   storageDays    60 non-null     int64 
 3   storageDesc    60 non-null     object
 4   storageMethod  60 non-null     object
dtypes: int64(1), object(4)
memory usage: 2.5+ KB


In [11]:
df.shape

(60, 5)

In [12]:
# 파서를 설정하고 프롬프트 템플릿에 지시사항을 주입합니다.
parser = PandasDataFrameOutputParser(dataframe=df)

# 파서의 지시사항을 출력합니다.
print(parser.get_format_instructions())

The output should be formatted as a string as the operation, followed by a colon, followed by the column or row to be queried on, followed by optional array parameters.
1. The column names are limited to the possible columns below.
2. Arrays must either be a comma-separated list of numbers formatted as [1,3,5], or it must be in range of numbers formatted as [0..4].
3. Remember that arrays are optional and not necessarily required.
4. If the column is not in the possible columns or the operation is not a valid Pandas DataFrame operation, return why it is invalid as a sentence starting with either "Invalid column" or "Invalid operation".

As an example, for the formats:
1. String "column:num_legs" is a well-formatted instance which gets the column num_legs, where num_legs is a possible column.
2. String "row:1" is a well-formatted instance which gets row 1.
3. String "column:num_legs[1,2]" is a well-formatted instance which gets the column num_legs for rows 1 and 2, where num_legs is a p

In [13]:
# 열 작업 예시입니다.
df_query = "name column을 조회해 주세요."


# 프롬프트 템플릿을 설정합니다.
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{question}\n",
    input_variables=["question"],  # 입력 변수 설정
    partial_variables={
        "format_instructions": parser.get_format_instructions()
    },  # 부분 변수 설정
)

# 체인 생성
chain = prompt | model | parser

# 체인 실행
parser_output = chain.invoke({"question": df_query})

# 출력
format_parser_output(parser_output)

{'name': {0: '두부',
          1: '콩나물',
          2: '숙주나물',
          3: '고구마',
          4: '감자',
          5: '양파',
          6: '마늘',
          7: '파',
          8: '생강',
          9: '오이',
          10: '가지',
          11: '호박',
          12: '옥수수',
          13: '상추',
          14: '깻잎',
          15: '쌈채소',
          16: '고추',
          17: '피망',
          18: '파프리카',
          19: '시금치',
          20: '부추',
          21: '나물',
          22: '양배추',
          23: '양상추',
          24: '브로콜리',
          25: '당근',
          26: '우엉',
          27: '연근',
          28: '마',
          29: '버섯',
          30: '배추',
          31: '무',
          32: '아스파라거스',
          33: '인삼',
          34: '더덕',
          35: '사과',
          36: '배',
          37: '감귤',
          38: '만감류',
          39: '수박',
          40: '멜론',
          41: '참외',
          42: '토마토',
          43: '딸기',
          44: '키위',
          45: '블루베리',
          46: '포도',
          47: '자두',
          48: '복숭아',
          49

In [14]:
df.head()

,category,name,storageDays,storageDesc,storageMethod
0,기타,두부,7,5~7일,물에 담가 밀폐용기에 넣어 냉장 보관. 매일 물을 갈아주면 더 오래 보관 가능합니다.
1,야채,콩나물,5,3~5일,씻지 않은 상태로 물기를 제거하고 밀폐용기에 담아 냉장 보관하세요. 빛을 차단하면 ...
2,야채,숙주나물,4,2~4일,"콩나물보다 쉽게 무르므로, 씻지 않고 밀폐용기에 담아 냉장 보관 후 가급적 빨리 소..."
3,야채,고구마,30,2~4주,10~15°C의 서늘하고 어두운 곳에 신문지로 감싸 보관하세요. (냉장 보관 시 맛...
4,야채,감자,30,2~4주,빛이 없는 서늘하고 통풍이 잘 되는 곳에 보관하세요. 사과와 함께 두면 싹 나는 것...


In [15]:
# row 0 ~ 4의 평균 소비기한을 구합니다.
df["storageDays"].head().mean()

15.2

In [16]:
# 임의의 Pandas DataFrame 작업 예시, 행의 수를 제한합니다.
df_query = "Retrieve the average of the storageDays from row 0 to 4."

# 체인 실행
parser_output = chain.invoke({"question": df_query})

# 결과 출력
print(parser_output)

{'mean': 15.2}


In [17]:
# 최대 소비기한을 구합니다.
df["storageDays"].max()

180

In [18]:
# The user query needs to be in the format expected by the PandasDataFrameOutputParser
df_query = "storageDays의 평균값을 알고싶어."

# 체인 실행
parser_output = chain.invoke({"question": df_query})

# 결과 출력
print(parser_output)

{'mean': 15.95}


In [19]:
fruit_count = df[df['category'] == '과일'].shape[0]
print(f"category가 '과일'인 데이터의 갯수: {fruit_count}")

category가 '과일'인 데이터의 갯수: 24


❓ 아래는 실패한 예시입니다.
PandasDataFrameOutputParser의 query 문법이 별도로 있는건지,
혹은 대응할 수 있는 query의 종류가 적은건지 좀 궁금합니다.

In [21]:
df_query = "df[df['category']=='과일'].shape[0]"
parser_output = chain.invoke({"question": df_query})
print(parser_output)

OutputParserException: Invalid operation: shape is not a valid Pandas DataFrame operation in this context.. Please check the format instructions.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 

In [ ]:
# The user query needs to be in the format expected by the PandasDataFrameOutputParser
df_query = "name이 고구마인 데이터의 storageMethod컬럼을 조회해 주세요."

# 체인 실행
parser_output = chain.invoke({"question": df_query})

# 결과 출력
print(parser_output)

OutputParserException: Invalid array format in 'storageMethod[name=='고구마']'.                     Please check the format instructions.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 

In [ ]:
# The user query needs to be in the format expected by the PandasDataFrameOutputParser
df_query = "name, storageDesc 컬럼을 조회해주세요. "

# 체인 실행
parser_output = chain.invoke({"question": df_query})

# 결과 출력
print(parser_output)

OutputParserException: Requested index name,storageDesc is out of bounds.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 